In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_clamp_loss
from dataclasses import dataclass
import torch

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    num_inds: int = 32
    dim_hidden: int = 128
    num_heads: int = 4
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.enc = nn.Sequential(
            ISAB(config.dim_context, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            nn.SiLU(),
            ISAB(config.dim_hidden, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            nn.SiLU(),
        )
        self.input_proj = nn.Linear(config.dim_input, config.dim_hidden)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.dec = nn.Sequential(
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
        )
        self.regr = nn.Sequential(
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        y = self.enc(context)           # [batch_size, context_size, dim_hidden]
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)            # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_hidden]
        y = self.regr(y)                # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_clamp_loss(y, labels, self.config.delta)
        return {'loss': loss, 'logits': y}

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_11_26_SILU'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_11_26_SILU-2024-11-26-13-32-22


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,

    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [6]:
trainer.train()
train_dataset.close()
val_dataset.close()

  0%|          | 0/150000 [00:00<?, ?it/s]

{'loss': 0.0328, 'grad_norm': 1.326286792755127, 'learning_rate': 4.999666666666667e-05, 'epoch': 0.0}
{'loss': 0.0156, 'grad_norm': 0.8520083427429199, 'learning_rate': 4.9993333333333335e-05, 'epoch': 0.0}
{'loss': 0.0102, 'grad_norm': 1.271254539489746, 'learning_rate': 4.999e-05, 'epoch': 0.0}
{'loss': 0.0084, 'grad_norm': 0.913791298866272, 'learning_rate': 4.9986666666666674e-05, 'epoch': 0.0}
{'loss': 0.0062, 'grad_norm': 0.2139165699481964, 'learning_rate': 4.998333333333334e-05, 'epoch': 0.0}
{'loss': 0.0089, 'grad_norm': 0.34764158725738525, 'learning_rate': 4.9980000000000006e-05, 'epoch': 0.0}
{'loss': 0.0075, 'grad_norm': 0.48827385902404785, 'learning_rate': 4.997666666666667e-05, 'epoch': 0.0}
{'loss': 0.0074, 'grad_norm': 0.2836458384990692, 'learning_rate': 4.997333333333333e-05, 'epoch': 0.0}
{'loss': 0.0075, 'grad_norm': 0.5590843558311462, 'learning_rate': 4.997e-05, 'epoch': 0.0}
{'loss': 0.0064, 'grad_norm': 0.41888150572776794, 'learning_rate': 4.996666666666667e

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.0030052883084863424, 'eval_runtime': 165.816, 'eval_samples_per_second': 1507.695, 'eval_steps_per_second': 37.692, 'epoch': 1.0}
{'loss': 0.0037, 'grad_norm': 0.1923009306192398, 'learning_rate': 4.374666666666667e-05, 'epoch': 1.0}
{'loss': 0.0031, 'grad_norm': 0.7382591962814331, 'learning_rate': 4.374333333333334e-05, 'epoch': 1.0}
{'loss': 0.0028, 'grad_norm': 0.28060388565063477, 'learning_rate': 4.3740000000000005e-05, 'epoch': 1.0}
{'loss': 0.0028, 'grad_norm': 0.7028530836105347, 'learning_rate': 4.373666666666667e-05, 'epoch': 1.0}
{'loss': 0.0022, 'grad_norm': 0.46020635962486267, 'learning_rate': 4.373333333333334e-05, 'epoch': 1.0}
{'loss': 0.0036, 'grad_norm': 0.12756432592868805, 'learning_rate': 4.373e-05, 'epoch': 1.0}
{'loss': 0.0024, 'grad_norm': 0.06363651156425476, 'learning_rate': 4.372666666666667e-05, 'epoch': 1.0}
{'loss': 0.0029, 'grad_norm': 0.0532558374106884, 'learning_rate': 4.3723333333333335e-05, 'epoch': 1.0}
{'loss': 0.0036, 'grad_norm'

  0%|          | 0/6250 [00:00<?, ?it/s]

...

{'eval_loss': 0.0019178555812686682, 'eval_runtime': 170.8258, 'eval_samples_per_second': 1463.479, 'eval_steps_per_second': 36.587, 'epoch': 6.0}
{'loss': 0.0022, 'grad_norm': 0.1767408847808838, 'learning_rate': 1.2496666666666668e-05, 'epoch': 6.0}
{'loss': 0.0019, 'grad_norm': 0.08807647973299026, 'learning_rate': 1.2493333333333333e-05, 'epoch': 6.0}
{'loss': 0.0021, 'grad_norm': 0.12183082848787308, 'learning_rate': 1.249e-05, 'epoch': 6.0}
{'loss': 0.0019, 'grad_norm': 0.17944379150867462, 'learning_rate': 1.2486666666666667e-05, 'epoch': 6.0}
{'loss': 0.002, 'grad_norm': 0.06674836575984955, 'learning_rate': 1.2483333333333335e-05, 'epoch': 6.0}
{'loss': 0.0018, 'grad_norm': 0.20720092952251434, 'learning_rate': 1.248e-05, 'epoch': 6.0}
{'loss': 0.0032, 'grad_norm': 0.3672357499599457, 'learning_rate': 1.2476666666666667e-05, 'epoch': 6.0}
{'loss': 0.0015, 'grad_norm': 0.0566425547003746, 'learning_rate': 1.2473333333333335e-05, 'epoch': 6.0}
{'loss': 0.0025, 'grad_norm': 0.211

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.001825681421905756, 'eval_runtime': 176.2106, 'eval_samples_per_second': 1418.757, 'eval_steps_per_second': 35.469, 'epoch': 7.0}
{'loss': 0.0014, 'grad_norm': 0.03253093734383583, 'learning_rate': 6.2466666666666664e-06, 'epoch': 7.0}
{'loss': 0.0031, 'grad_norm': 0.2373724728822708, 'learning_rate': 6.243333333333333e-06, 'epoch': 7.0}
{'loss': 0.0015, 'grad_norm': 0.48140525817871094, 'learning_rate': 6.24e-06, 'epoch': 7.0}
{'loss': 0.0018, 'grad_norm': 0.1128675565123558, 'learning_rate': 6.236666666666667e-06, 'epoch': 7.0}
{'loss': 0.0016, 'grad_norm': 0.13398928940296173, 'learning_rate': 6.2333333333333335e-06, 'epoch': 7.0}
{'loss': 0.0013, 'grad_norm': 0.2772919237613678, 'learning_rate': 6.2300000000000005e-06, 'epoch': 7.0}
{'loss': 0.0013, 'grad_norm': 0.08761367201805115, 'learning_rate': 6.226666666666667e-06, 'epoch': 7.0}
{'loss': 0.0019, 'grad_norm': 0.1212082952260971, 'learning_rate': 6.223333333333334e-06, 'epoch': 7.0}
{'loss': 0.0014, 'grad_norm'

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.0017758607864379883, 'eval_runtime': 197.4891, 'eval_samples_per_second': 1265.892, 'eval_steps_per_second': 31.647, 'epoch': 8.0}
{'train_runtime': 8406.0808, 'train_samples_per_second': 713.769, 'train_steps_per_second': 17.844, 'train_loss': 0.0024277304469856125, 'epoch': 8.0}


In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)

In [8]:
from src.visualization.generate_mesh import generate_mesh
from src.data.load_data import get_data_dir

obj_path = get_data_dir() / 'intermediate' / 'bunny' / 'stanford-bunny.obj'
infered_obj_path = result_dir / f"bunny-{current_date}-generalization.obj"
generate_mesh(model, obj_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [9]:
hdf5_path = get_data_dir() / 'processed' / 'bunny' / 'stanford-bunny_train.hdf5'
infered_obj_path = result_dir / f"{current_date}-train.obj"
generate_mesh(model, hdf5_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]